<a href="https://colab.research.google.com/github/eli-osherovich/mnist-demo/blob/main/mnist_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist, fashion_mnist

In [2]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-536b977f-9ddc-d07c-5988-f641eac1bd2a)


In [3]:
# the data, split between train and test sets
dataset = fashion_mnist
dataset = mnist


(x_train, y_train), (x_test, y_test) = dataset.load_data()

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

x_train = x_train.astype(np.float32)/255
x_test = x_test.astype(np.float32)/255
y_train = y_train.astype(np.int)
y_test = y_test.astype(np.int)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)


In [4]:
def gen_model(activation="relu"):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.Input(shape=(28, 28, 1)))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(10))
  
  return model


def compile_model(model, lr=0.001):
  model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=keras.optimizers.Adam(learning_rate=lr),
                metrics=['accuracy'])
  

early_stop = tf.keras.callbacks.EarlyStopping(
      monitor='val_accuracy',
      patience=20,
      restore_best_weights=True,
)

In [5]:
%%time
model = gen_model()
compile_model(model)
model.fit(x_train, y_train,
          batch_size=128,
          epochs=1000,
          verbose=1,
          callbacks=[early_stop],
          validation_data=(x_test, y_test),
          validation_batch_size=2048)

Epoch 1/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.6740 - accuracy: 0.8327 - val_loss: 0.3812 - val_accuracy: 0.9025
Epoch 2/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.3619 - accuracy: 0.9017 - val_loss: 0.3203 - val_accuracy: 0.9132
Epoch 3/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.3197 - accuracy: 0.9122 - val_loss: 0.2985 - val_accuracy: 0.9168
Epoch 4/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.2999 - accuracy: 0.9168 - val_loss: 0.2862 - val_accuracy: 0.9190
Epoch 5/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.2879 - accuracy: 0.9198 - val_loss: 0.2781 - val_accuracy: 0.9220
Epoch 6/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.2800 - accuracy: 0.9216 - val_loss: 0.2739 - val_accuracy: 0.9233
Epoch 7/1000
469/469 [==============================] - 1s 2ms/step - loss: 0.2739 - accuracy: 0.9236 - val_loss: 0.2704 - val_accuracy:

In [6]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 0.26179051399230957
Test accuracy: 0.9290000200271606
